# Rock Paper Scissors
rock paper scissors with stupid NN T_T

`Author: kafri8889 (Anaf Naufalian)`
[Github](https://github.com/kafri8889/)

In [2]:
# @author: Anaf Naufalian (kafri8889)

import random
import numpy as np
import tensorflow as tf

In [3]:
# @author: Anaf Naufalian (kafri8889)

# Digunakan untuk menyimpan history inputan yang akan digunakan untuk melatih model
# contoh isinya: [[1, 0, 0], [0, 1, 0]]
history_inputs = []
# Digunakan untuk menyimpan history output dari inputan yang akan digunakan untuk melatih model
# contoh isinya: [[0, 0, 1], [1, 0, 0]]
history_outputs = []

In [4]:
# @author: Anaf Naufalian (kafri8889)

# Fungsi ini digunakan untuk mengubah pilihan (rock, paper, scissors) menjadi vektor berdimensi 1x3
def rps_to_vector(choice: str):
    if choice == 'rock':
        return [1, 0, 0]
    elif choice == 'paper':
        return [0, 1, 0]
    
    return [0, 0, 1]

In [5]:
# @author: Anaf Naufalian (kafri8889)

# Generate training dataset
def generate_dataset(num_of_games):
    x = []
    y = []

    actions = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])  # rock, paper, scissors

    for _ in range(num_of_games):
        # Ambil pilihan acak (user)
        user_action = np.random.randint(3)
        user_choice = actions[np.random.randint(3)]

        # Ambil pilihan acak (computer)
        computer_action = np.random.randint(3)
        computer_choice = actions[computer_action]

        # jika pilihan user dan komputer sama
        if np.array_equal(user_choice, computer_choice):
            outcome = [0, 0, 1]  # seri (scissors)
        elif (user_action == 0 and computer_action == 2) or (user_action == 1 and computer_action == 0) or (user_action == 2 and computer_action == 1):
            outcome = [1, 0, 0]  # user menang (rock)
        else:
            outcome = [0, 1, 0]  # computer menang (paper)

        # menambahkan ke dataset
        x.append(user_choice)
        y.append(outcome)

    return np.array(x), np.array(y)

In [6]:
# @author: Anaf Naufalian (kafri8889)

def create_model():
    # membuat model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation="relu", input_shape=(3,)),
        tf.keras.layers.Dense(64, activation="relu"),
        # kenapa menggunakan softmax?, karena bot akan memilih 1 dari 3 pilihan, jika hanya 2 pilihan, kita bisa menggunakan 'sigmoid'
        tf.keras.layers.Dense(3, activation="softmax") # 3 node (rock, paper, or scissors)
    ])

    # mengapa menggunakan loss categorical cross entropy?, karena kita akan melakukan multi class classification, atau kita
    # akan mengklasifikasi lebih dari 2 tipe, yaitu rock, paper, dan scissors
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

    # jika model sudah dibuat dan dilatih, return
    return model

In [7]:
# @author: Anaf Naufalian (kafri8889)

def train_model():
    # melatih model
    # verbose = 0, silent training
    model.fit(np.array(history_inputs), np.array(history_outputs), epochs=10, batch_size=32, verbose=0)

In [16]:
# @author: Anaf Naufalian (kafri8889)

def pre_trained_model():
    # model yang sudah dilatih menggunakan data yang di generate
    x, y = generate_dataset(1000)

    model = create_model()
    model.fit(np.array(x), np.array(y), epochs=20, batch_size=16, verbose=0)
    return model

In [17]:
# @author: Anaf Naufalian (kafri8889)

pre_trained_model = pre_trained_model()

In [19]:
# @author: Anaf Naufalian (kafri8889)

# membuat model pertama kali
model = create_model()

print("Welcome to Rock, Paper, Scissors!")
print("Before start, what model yang ingin kamu gunakan?")
print("1. Pre trained model")
print("2. Untrained model")
selected_model = input("Select model (1 or 2): ")
print()

if selected_model == "1":
    model = pre_trained_model
    print("You choose pre trained model.")
else:
    print("You choose untrained model")

print()

user_wins = 0
computer_wins = 0

choices = {
  "1": "rock",
  "2": "paper",
  "3": "scissors"
}

while True:
  
  if len(history_inputs) > 0:
      last_input = np.array([history_inputs[-1]])
  else:
      last_input = np.array([rps_to_vector(choices[str(random.randint(1, 3))])])

  # bot akan memilih berdasarkan prediksi dari model
  predicted = model.predict(last_input)
  cpu_choice = choices[str(np.argmax(predicted))]

  user_choice = input("(1) Rock, (2) Paper, or (3) Scissors? Select number!")
  while user_choice not in choices:
    user_choice = input("That is not a valid choice. Please try again: ")
      
  user_choice = choices[user_choice]

  print()
  print("Your choice:", user_choice)
  print("Computer's choice:", cpu_choice)
  print()

  if user_choice == 'rock':
    if cpu_choice == 'rock':
      print("It's a tie!")
    elif cpu_choice == 'scissors':
      print("You win!")
      user_wins+=1
    elif cpu_choice == 'paper':
      print("You lose!")
      computer_wins+=1
  elif user_choice == 'paper':
    if cpu_choice == 'paper':
      print("It's a tie!")
    elif cpu_choice == 'rock':
      print("You win!")
      user_wins+=1
    elif cpu_choice == 'scissors':
      print("You lose!")
      computer_wins+=1
  elif user_choice == 'scissors':
    if cpu_choice == 'scissors':
      print("It's a tie!")
    elif cpu_choice == 'paper':
      print("You win!")
      user_wins+=1
    elif cpu_choice == 'rock':
      print("You lose!")
      computer_wins+=1

  print()
  print("You have "+str(user_wins)+" wins")
  print("The computer has "+str(computer_wins)+" wins")
  print()
    
  # Menyimpan input dan output ke dalam history untuk data pelatihan model
  history_inputs.append(rps_to_vector(user_choice))
  history_outputs.append(rps_to_vector(cpu_choice))
    
  # latih ulang model, jika user memilih untrained model
  if selected_model == 2:
      train_model()

  repeat = input("Play again? (Y/N) ").lower()
  while repeat not in ['y', 'n']:
    repeat = input("That is not a valid choice. Please try again: ").lower()

  if repeat == 'n':
    print("Bye bye")
    break

  print("\n----------------------------\n")


Welcome to Rock, Paper, Scissors!
Before start, what model yang ingin kamu gunakan?
1. Pre trained model
2. Untrained model
Select model (1 or 2):  2

You choose untrained model

1/1 [==============================] - 0s 60ms/step
(1) Rock, (2) Paper, or (3) Scissors? Select number! 1

Your choice: rock
Computer's choice: paper

You lose!

You have 0 wins
The computer has 1 wins

Play again? (Y/N)  y

----------------------------

1/1 [==============================] - 0s 20ms/step
(1) Rock, (2) Paper, or (3) Scissors? Select number! 2

Your choice: paper
Computer's choice: paper

It's a tie!

You have 0 wins
The computer has 1 wins

Play again? (Y/N)  y

----------------------------

1/1 [==============================] - 0s 21ms/step
(1) Rock, (2) Paper, or (3) Scissors? Select number! 1

Your choice: rock
Computer's choice: paper

You lose!

You have 0 wins
The computer has 2 wins

Play again? (Y/N)  y

----------------------------

1/1 [==============================] - 0s 21ms/ste